Most tokenization schemes have two parts: a token learner, and a token seg-menter. The token learner takes a raw training corpus (sometimes roughly pre
separated into words, for example by whitespace) and induces a vocabulary, a set
 of tokens. The token segmenter takes a raw test sentence and segments it into the
 tokens in the vocabulary.

1. Byte-Pair Encoding


The BPE token learner begins
 with a vocabulary that is just the set of all individual characters. It then examines the
 training corpus, chooses the two symbols that are most frequently adjacent (say ‘A’,
 ‘B’), adds a new merged symbol ‘AB’ to the vocabulary, and replaces every adjacent
 ’A’ ’B’ in the corpus with the new ‘AB’. It continues to count and merge, creating
 new longer and longer character strings, until k merges have been done creating
 k novel tokens; k is thus a parameter of the algorithm. The resulting vocabulary
 consists of the original set of characters plus k new symbols.

Below is the example that walks through creating a vocabulary, counting pair frequencies, and iteratively merging the most frequent pairs.

In [5]:
# Sample corpus: a list of words.

corpus = ["low_", "lower_", "lowest_", "new_", "newer_", "newest_", "wide_", "wider_", "widest_"]

In [17]:
# to retrieve unique chars
def get_unique_chars(corpus):
    unique_chars = set()

    # iterate over each word in the corpus
    for word in corpus:
        unique_chars.update(list(word))
    return sorted(unique_chars)

In [47]:

# sort the retrieved characters
sorted_unique_chars = get_unique_chars(corpus)

print("Unique Characters:", sorted_unique_chars)

Unique Characters: ['_', 'd', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w']


In [46]:
# Create initial vocabulary:
# Each word is represented as a sequence of characters separated by spaces.
vocab = {" ".join(word): 1 for word in corpus}
print("Initial Vocabulary:")
for word, freq in vocab.items():
    print(f"{word} : {freq}")

Initial Vocabulary:
l o w _ : 1
l o w e r _ : 1
l o w e s t _ : 1
n e w _ : 1
n e w e r _ : 1
n e w e s t _ : 1
w i d e _ : 1
w i d e r _ : 1
w i d e s t _ : 1


In [48]:
def get_stats(vocab, allowed_tokens):
    """
    Count frequency of adjacent symbol pairs in the vocabulary.
    but only count pairs where both symbols are in allowed_tokens.

    vocab: dict mapping "words" (as strings of space-separated symbols) to frequency.
    allowed_tokens: a set (or list) of tokens that are allowed (e.g., sorted unique characters).

    Returns a dictionary with keys as tuple (symbol1, symbol2) and values as counts.
    """
    pairs = {}
    for word, freq in vocab.items():
        symbols = word.split()
        # Count each adjacent pair in the word
        for i in range(len(symbols) - 1):
            if symbols[i] in allowed_tokens and symbols[i + 1] in allowed_tokens:
                pair = (symbols[i], symbols[i + 1])
                pairs[pair] = pairs.get(pair, 0) + freq
    return pairs

In [49]:
# Perform a single example merge iteration using the modified get_stats.
allowed_tokens = set(sorted_unique_chars)
pairs = get_stats(vocab, allowed_tokens )
print("\nPair frequencies (only counting pairs with allowed tokens):")
for pair, count in pairs.items():
    print(f"{pair}: {count}")


Pair frequencies (only counting pairs with allowed tokens):
('l', 'o'): 3
('o', 'w'): 3
('w', '_'): 2
('w', 'e'): 4
('e', 'r'): 3
('r', '_'): 3
('e', 's'): 3
('s', 't'): 3
('t', '_'): 3
('n', 'e'): 3
('e', 'w'): 3
('w', 'i'): 3
('i', 'd'): 3
('d', 'e'): 3
('e', '_'): 1


In [42]:
def merge_vocab(pair, vocab):
    """
    Merge all occurrences of the given pair in the vocabulary.
    pair: a tuple (symbol1, symbol2) to be merged.
    vocab: the current vocabulary dictionary.
    Returns a new vocabulary with the merged pair.
    """
    new_vocab = {}
    # Create the merged token
    merged_token = ''.join(pair)
    # Define a string pattern that represents the pair separated by a space
    bigram = ' '.join(pair)
    
    for word, freq in vocab.items():
        # Replace the pair with the merged token in the word.
        new_word = word.replace(bigram, merged_token)
        new_vocab[new_word] = freq
    return new_vocab

In [51]:
# Number of merge operations
num_merges = 15       # a parameter of BPE algo

print("\nMerging steps:")

# Perform iterative merging.
for i in range(num_merges):

  # Compute pair frequencies only for pairs where both tokens are allowed.
  pairs = get_stats(vocab, allowed_tokens)
  if not pairs:
    break

  # Select the most frequent pair.
  best_pair = max(pairs, key=pairs.get)

  vocab = merge_vocab(best_pair, vocab)
  merged_token = ''.join(best_pair)
  allowed_tokens.add(merged_token)

    # For output, show the merged pair and the updated sorted allowed tokens.
  output_line = f"({best_pair[0]},{best_pair[1]}) ,"
  output_line += ",".join(sorted(allowed_tokens))
  print(output_line)



Merging steps:
(lo,we) ,_,d,e,i,l,lo,lowe,n,ne,o,r,r_,s,st,st_,t,w,w_,we,wi,wid,wide
(ne,we) ,_,d,e,i,l,lo,lowe,n,ne,newe,o,r,r_,s,st,st_,t,w,w_,we,wi,wid,wide
(lo,w_) ,_,d,e,i,l,lo,low_,lowe,n,ne,newe,o,r,r_,s,st,st_,t,w,w_,we,wi,wid,wide
(lowe,r_) ,_,d,e,i,l,lo,low_,lowe,lower_,n,ne,newe,o,r,r_,s,st,st_,t,w,w_,we,wi,wid,wide
(lowe,st_) ,_,d,e,i,l,lo,low_,lowe,lower_,lowest_,n,ne,newe,o,r,r_,s,st,st_,t,w,w_,we,wi,wid,wide
(ne,w_) ,_,d,e,i,l,lo,low_,lowe,lower_,lowest_,n,ne,new_,newe,o,r,r_,s,st,st_,t,w,w_,we,wi,wid,wide
(newe,r_) ,_,d,e,i,l,lo,low_,lowe,lower_,lowest_,n,ne,new_,newe,newer_,o,r,r_,s,st,st_,t,w,w_,we,wi,wid,wide
(newe,st_) ,_,d,e,i,l,lo,low_,lowe,lower_,lowest_,n,ne,new_,newe,newer_,newest_,o,r,r_,s,st,st_,t,w,w_,we,wi,wid,wide
(wide,_) ,_,d,e,i,l,lo,low_,lowe,lower_,lowest_,n,ne,new_,newe,newer_,newest_,o,r,r_,s,st,st_,t,w,w_,we,wi,wid,wide,wide_
(wide,r_) ,_,d,e,i,l,lo,low_,lowe,lower_,lowest_,n,ne,new_,newe,newer_,newest_,o,r,r_,s,st,st_,t,w,w_,we,wi,wid,wide,wide_,

2. Word Piece

It’s very similar to BPE in terms of the training, but the actual tokenization is done differently.   Like BPE, WordPiece starts from a small vocabulary including the special tokens used by the model and the initial alphabet. Since it identifies subwords by adding a prefix (like ## for BERT), each word is initially split by adding that prefix to all the characters inside the word.

Then, again like BPE, WordPiece learns merge rules. The main difference is the way the pair to be merged is selected. Instead of selecting the most frequent pair, WordPiece computes a score for each pair, using the following formula:  
   score = (freq_of_pair) / (freq_of_first_element × freq_of_second_element)


In [69]:
def initialize_corpus(corpus):
    """
    Given a list of words, tokenize each word into a list of individual characters.
    Returns:
        vocab: a list of token lists, one per word.
        global_vocab: a set containing all tokens (initial characters) plus [UNK].
    """
    tokenized_corpus = []
    global_vocab = set()
    for word in corpus:
        tokens = list(word)
        tokenized_corpus.append(tokens)
        global_vocab.update(tokens)
    global_vocab.add("[UNK]")  # Add unknown token
    return tokenized_corpus, global_vocab

def get_token_freq(tokenized_corpus):
    """
    Count frequencies of individual tokens in the current corpus.
    """
    freq = {}
    for tokens in tokenized_corpus:
        for token in tokens:
            freq[token] = freq.get(token, 0) + 1
    return freq

def get_pair_freq(tokenized_corpus):
    """
    Count frequencies of adjacent token pairs in the corpus.
    Returns:
        A dictionary mapping a tuple (token1, token2) to its frequency.
    """
    pair_freq = {}
    for tokens in tokenized_corpus:
        for i in range(len(tokens) - 1):
            pair = (tokens[i], tokens[i+1])
            pair_freq[pair] = pair_freq.get(pair, 0) + 1
    return pair_freq

def select_best_pair(pair_freq, token_freq):
    """
    Evaluate candidate merges using the scoring function:
      score = P(xy) / (P(x)*P(y))
    Returns the best pair (tuple) and its score.
    """
    best_pair = None
    best_score = -1.0
    for pair, freq in pair_freq.items():
        # Use frequencies as proxies for probability counts.
        p_x = token_freq.get(pair[0], 1)
        p_y = token_freq.get(pair[1], 1)
        score = freq / (p_x * p_y)
        if score > best_score:
            best_score = score
            best_pair = pair
    return best_pair, best_score

def merge_corpus(best_pair, tokenized_corpus):
    """
    Merge the best pair in every word in the corpus.
    For each occurrence of the pair:
       - if the pair occurs at the beginning of the word, merge without a prefix;
       - otherwise, add a "##" prefix to the merged token.
    Returns the updated corpus and a set of newly created merged tokens.
    """
    new_tokens_set = set()
    merged_token_plain = "".join(best_pair)  # unprefixed version
    new_corpus = []
    
    for tokens in tokenized_corpus:
        new_tokens = []
        i = 0
        while i < len(tokens):
            # Check if we can merge the next two tokens.
            if i < len(tokens) - 1 and (tokens[i], tokens[i+1]) == best_pair:
                # If this merge is at the start of the word, do not prefix.
                if i == 0:
                    new_token = merged_token_plain
                else:
                    new_token = "##" + merged_token_plain
                new_tokens.append(new_token)
                new_tokens_set.add(new_token)
                i += 2
            else:
                new_tokens.append(tokens[i])
                i += 1
        new_corpus.append(new_tokens)
    return new_corpus, new_tokens_set

# -------------------------------
# Main WordPiece Training Procedure
# -------------------------------

# Sample training corpus (list of words)
corpus = ["low", "lower", "newest", "widest"]

# Initialize the corpus and global vocabulary (starting with individual characters)
tokenized_corpus, global_vocab = initialize_corpus(corpus)

# Set training parameters:
max_vocab_size = 50        # desired maximum vocabulary size
score_threshold = 0.1      # minimum score threshold to accept a merge
max_merges = 100           # maximum number of merge iterations

print("Initial global vocabulary:")
print(sorted(global_vocab))
print("\nStarting WordPiece training...\n")

for merge_iter in range(max_merges):
    token_freq = get_token_freq(tokenized_corpus)
    pair_freq = get_pair_freq(tokenized_corpus)
    
    if not pair_freq:
        break
    
    best_pair, best_score = select_best_pair(pair_freq, token_freq)
    
    # Stop if the best score is below the threshold
    if best_score < score_threshold:
        print(f"Stopping: best merge score {best_score:.4f} fell below the threshold {score_threshold}.")
        break
    
    # Stop if the global vocabulary has reached the desired size.
    if len(global_vocab) >= max_vocab_size:
        print(f"Stopping: reached maximum vocabulary size of {max_vocab_size}.")
        break
    
    # Merge the best pair in the corpus.
    tokenized_corpus, new_tokens = merge_corpus(best_pair, tokenized_corpus)
    
    # Update the global vocabulary with the new token(s).
    global_vocab.update(new_tokens)
    
    print(f"Merge {merge_iter+1}: Merged pair {best_pair} ->", end=" ")
    # Print merged tokens (could be both with and without prefix depending on position)
    print(", ".join(sorted(new_tokens)), f"with score {best_score:.4f}")
    print("Global vocabulary size:", len(global_vocab))
    print("Current global vocabulary:")
    print(", ".join(sorted(global_vocab)))
    print("-" * 60)

print("\nFinal global vocabulary:")
print(", ".join(sorted(global_vocab)))


Initial global vocabulary:
['[UNK]', 'd', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w']

Starting WordPiece training...

Merge 1: Merged pair ('i', 'd') -> ##id with score 1.0000
Global vocabulary size: 12
Current global vocabulary:
##id, [UNK], d, e, i, l, n, o, r, s, t, w
------------------------------------------------------------
Merge 2: Merged pair ('l', 'o') -> lo with score 0.5000
Global vocabulary size: 13
Current global vocabulary:
##id, [UNK], d, e, i, l, lo, n, o, r, s, t, w
------------------------------------------------------------
Merge 3: Merged pair ('s', 't') -> ##st with score 0.5000
Global vocabulary size: 14
Current global vocabulary:
##id, ##st, [UNK], d, e, i, l, lo, n, o, r, s, t, w
------------------------------------------------------------
Merge 4: Merged pair ('lo', 'w') -> low with score 0.2500
Global vocabulary size: 15
Current global vocabulary:
##id, ##st, [UNK], d, e, i, l, lo, low, n, o, r, s, t, w
------------------------------------------------------

Flow of Word Piece:  

<img src="C:\Shilpi\BFH\MSE_Courses\Semester3\TSM_AdvNLP\Lec1_Intro\Word Piece.png" alt="Sample Image" width="150"> 
 
https://medium.com/@atharv6f_47401/wordpiece-tokenization-a-bpe-variant-73cc48865cbf

1. I/P coupus -> initialize vocab with charac -> Add special tikens UNK, ## -> split corpus into words -> generate all possible subword combinatons -> calculate token frequencies -> Iterative merging process   
2. check if vocab size is reached or Score below threshold  
3. If yes, finalise vocab ->tokenize of new Text -> split into words -> for each word -> match longest subword from Start -> if match found..  
yes -> add to o/p -> check for more characters -> (if yes -> again match longest subword from Start, and continue again.) (if no, check for more words -> )  
no -> add UNK token -> 

Key Differences between BPE and WordPiece  
Initialization  
BPE: Initializes the vocabulary with individual characters or bytes.  
WordPiece: Starts with characters plus special tokens, specifically UNK (unknown) and HASH (representing ##) for non-initial subwords.  

Subword Generation  
BPE: Directly counts the frequencies of character pairs in the corpus.  
WordPiece: Generates all possible subword combinations before scoring.  

Merging Criterion    
BPE: Selects and merges the most frequent pair in each iteration.  
WordPiece: Calculates scores for pairs and selects the highest scoring pair for merging.  

Special Token Usage  
BPE: Does not use special tokens in its base algorithm.  
WordPiece: Utilizes the HASH prefix (##) for non-word-initial subwords, maintaining word boundary information.  

Stopping Criterion  
BPE: Typically stops when a predefined vocabulary size is reached.  
WordPiece: Can stop based on vocabulary size or when scores fall below a certain threshold.  

Tokenization of New Text  
BPE: Applies learned merge rules sequentially.  
WordPiece: Uses a longest match approach, incorporating UNK for unknown words and HASH for non-initial subwords.  

#### 3.4 Hugging Face Implementations

https://colab.research.google.com/github/huggingface/notebooks/blob/master/course/en/chapter6/section5.ipynb   

https://colab.research.google.com/github/huggingface/notebooks/blob/master/course/en/chapter6/section6.ipynb

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

In [ ]:
pip install transformers

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

c:\Users\itssh\anaconda\envs\AdvNLP\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [ ]:
from collections import defaultdict

# creating default dictionary of corpus generated at the start of this notebook
word_freqs = defaultdict(int)
for text in corpus:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]
    for word in new_words:
        word_freqs[word] += 1

word_freqs

defaultdict(int,
            {'low': 1,
             '_': 9,
             'lower': 1,
             'lowest': 1,
             'new': 1,
             'newer': 1,
             'newest': 1,
             'wide': 1,
             'wider': 1,
             'widest': 1})

In [ ]:
# generating characters with adding suffix ##
alphabet = []
for word in word_freqs.keys():
    if word[0] not in alphabet:
        alphabet.append(word[0])
    for letter in word[1:]:
        if f"##{letter}" not in alphabet:
            alphabet.append(f"##{letter}")

alphabet.sort()
alphabet

print(alphabet)

['##d', '##e', '##i', '##o', '##r', '##s', '##t', '##w', '_', 'l', 'n', 'w']


In [ ]:
#adding an unkown token to the  characters list
vocab = ["[UNK]"] + alphabet.copy()

In [ ]:
# to tokenize each word into subword units
splits = {
    word: [c if i == 0 else f"##{c}" for i, c in enumerate(word)]
    for word in word_freqs.keys()
}

In [ ]:
# function to compute pair scores
def compute_pair_scores(splits):
    letter_freqs = defaultdict(int)
    pair_freqs = defaultdict(int)
    for word, freq in word_freqs.items():
        split = splits[word]
        if len(split) == 1:
            letter_freqs[split[0]] += freq
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            letter_freqs[split[i]] += freq
            pair_freqs[pair] += freq
        letter_freqs[split[-1]] += freq

    scores = {
        pair: freq / (letter_freqs[pair[0]] * letter_freqs[pair[1]])
        for pair, freq in pair_freqs.items()
    }
    return scores

In [ ]:
# calculating pair scores
pair_scores = compute_pair_scores(splits)
for i, key in enumerate(pair_scores.keys()):
    print(f"{key}: {pair_scores[key]}")
    if i >= 15:
        break

('l', '##o'): 0.3333333333333333
('##o', '##w'): 0.16666666666666666
('##w', '##e'): 0.06666666666666667
('##e', '##r'): 0.1
('##e', '##s'): 0.1
('##s', '##t'): 0.3333333333333333
('n', '##e'): 0.1
('##e', '##w'): 0.05
('w', '##i'): 0.3333333333333333
('##i', '##d'): 0.3333333333333333
('##d', '##e'): 0.1


In [38]:
# checking the best pair score
best_pair = ""
max_score = None
for pair, score in pair_scores.items():
    if max_score is None or max_score < score:
        best_pair = pair
        max_score = score

print(best_pair, max_score)

('l', '##o') 0.3333333333333333


In [41]:
# merging pairs as per score order
print("Using merge_pair defined here.")
def merge_pair(a, b, splits):
    for word in word_freqs:
        split = splits[word]
        if len(split) == 1:
            continue
        i = 0
        while i < len(split) - 1:
            if split[i] == a and split[i + 1] == b:
                merge = a + b[2:] if b.startswith("##") else a + b
                split = split[:i] + [merge] + split[i + 2 :]
            else:
                i += 1
        splits[word] = split
    return splits
print("merge_pair was called successfully.")

Using merge_pair defined here.
merge_pair was called successfully.


In [40]:
# checking if best_pair is tuple
print(f"best_pair: {best_pair}, type: {type(best_pair)}")


best_pair: ('l', '##o'), type: <class 'tuple'>


In [42]:
merge_pair('l', '##o', splits)

{'low': ['low'],
 '_': ['_'],
 'lower': ['lower'],
 'lowest': ['lowest'],
 'new': ['new'],
 'newer': ['newer'],
 'newest': ['newest'],
 'wide': ['wide'],
 'wider': ['wider'],
 'widest': ['widest']}

In [50]:
# running and generating the merged pairs
vocab_size = 70
while len(vocab) < vocab_size:
    scores = compute_pair_scores(splits)

    best_pair, max_score = (None, None), None
    for pair, score in scores.items():
        if max_score is None or max_score < score:
            best_pair = pair
            max_score = score

    splits = merge_pair(*best_pair, splits) 

    if best_pair[1].startswith("##") and len(best_pair[1]) > 2:
        new_token = best_pair[0] + best_pair[1][2:]
    else:
        new_token = best_pair[0] + best_pair[1]

    vocab.append(new_token)

AttributeError: 'NoneType' object has no attribute 'startswith'

In [47]:
print(vocab)

['[UNK]', '##d', '##e', '##i', '##o', '##r', '##s', '##t', '##w', '_', 'l', 'n', 'w', 'lo', '##st', 'wi', 'wid', 'low', '##er', '##est', 'ne', 'new', 'wide', 'wider', 'widest', 'lower', 'newer', 'lowest', 'newest']


Question: I have issue with merge_pair functon which is not clearly generating me the pairs. Please guide me to the issue happeining here.